In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import zipfile

# Define a function to create and fine-tune the model

In [49]:
# Setup directory paths
train_dir = 'content/normal'  # Update with your path
test_dir = 'content/test'  # Update with your path
val_dir = 'content/validation'       # Update with your path

# Prepare ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2, height_shift_range=0.2,
                                   shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Define a function to create and fine-tune the model
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

models = {
    'VGG16': VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'ResNet50': ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3)),
    'InceptionV3': InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
}

# Fine-tuning and training models
for name, base_model in models.items():
    print(f"Training and evaluating {name}")
    model = build_model(base_model)
    history = model.fit(train_generator, validation_data=validation_generator, epochs=10, verbose=1)
    model.save(f'{name}_breast_cancer.h5')

    # Plotting training history
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{name} Training and Validation Accuracy')
    plt.legend()
    plt.show()

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', shuffle=False)

model = tf.keras.models.load_model('VGG16_breast_cancer.h5')

# Predictions and ground truth
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

# Confusion matrix and classification report
cm = confusion_matrix(true_classes, predicted_classes)
print('Confusion Matrix')
print(cm)
print('\nClassification Report')
cr = classification_report(true_classes, predicted_classes, target_names=test_generator.class_indices.keys())
print(cr)

# Calculating TP, TN, FP, FN
TP = np.diag(cm)
FP = cm.sum(axis=0) - TP
FN = cm.sum(axis=1) - TP
TN = cm.sum() - (FP + FN + TP)

print(f'True Positives: {TP}')
print(f'True Negatives: {TN}')


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Training and evaluating VGG16


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [38]:
!unzip test.zip


Archive:  test.zip
   creating: normal/
replace __MACOSX/._normal? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: __MACOSX/._normal       
  inflating: normal/normal (54)_mask.png  
  inflating: __MACOSX/normal/._normal (54)_mask.png  
  inflating: normal/normal (52).png  
  inflating: __MACOSX/normal/._normal (52).png  
  inflating: normal/normal (44).png  
  inflating: __MACOSX/normal/._normal (44).png  
  inflating: normal/normal (108).png  
  inflating: __MACOSX/normal/._normal (108).png  
  inflating: normal/normal (13).png  
  inflating: __MACOSX/normal/._normal (13).png  
  inflating: normal/normal (89)_mask.png  
  inflating: __MACOSX/normal/._normal (89)_mask.png  
  inflating: normal/normal (40)_mask.png  
  inflating: __MACOSX/normal/._normal (40)_mask.png  
  inflating: normal/normal (87).png  
  inflating: __MACOSX/normal/._normal (87).png  
  inflating: normal/normal (68).png  
  inflating: __MACOSX/normal/._normal (68).png  
  inflating: normal/normal (124).png  
 

In [25]:
dataset_base = 'normal'
  # Update the path as necessary
if os.path.exists(dataset_base):
    images = [img for img in os.listdir(dataset_base) if img.endswith(('.png', '.jpg', '.jpeg'))]
    print(f"Total images found: {len(images)}")
else:
    print("Directory not found")


Total images found: 266


In [42]:


datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow_from_directory(
    'content/normal',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = datagen.flow_from_directory(
    'content/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [43]:


def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)  # Change for binary output
    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [47]:
# Fine-tuning and training models
results = {}
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)

val_datagen = ImageDataGenerator(rescale=1./255)

train_path = 'content/normal'  # Adjust this path
val_path = 'content/validation'
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'  # Change this based on your model's output (binary or categorical)
)

validation_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

for name, base_model in models.items():
    print(f"Training and evaluating {name}")
    model = build_model(base_model)



    # Evaluate the model
    eval_result = model.evaluate(validation_generator)
    results[name] = {
        'Accuracy': eval_result[1],
        'Loss': eval_result[0]
    }

    # Plotting training history
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.title(f'{name} Training and Validation Accuracy')
    plt.legend()
    plt.show()

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Training and evaluating VGG16


ValueError: Asked to retrieve element 0, but the Sequence has length 0

In [ ]:
# Summary of results
print("Model performance summary:")
for model_name, metrics in results.items():
    print(f"{model_name} - Accuracy: {metrics['Accuracy']:.2f}, Loss: {metrics['Loss']:.2f}")

# Save models
for model_name in models:
    models[model_name].save(f'{model_name}_breast_cancer.h5')